## 3. 데이터 전처리 (16kHz 변환, mono 변환, max_length 계산)

In [ ]:
!pip install pydub librosa soundfile

(1) 전처리 함수 선언

In [1]:
from pathlib import Path
import os
import librosa
import soundfile as sf
import csv

def preprocess_wav_files_without_padding(phishing_wav_dir: Path, normal_wav_dir: Path,
                                          output_dir: Path, output_csv_path: Path,
                                          sample_rate: int = 16000):
    # 📁 출력 디렉토리 및 메타데이터 저장 디렉토리 생성
    output_dir.mkdir(parents=True, exist_ok=True)
    output_csv_path.parent.mkdir(parents=True, exist_ok=True)

    # 🔍 입력 디렉토리 정보 출력
    print(f"📂 phishing 디렉토리: {phishing_wav_dir}")
    print(f"📂 normal 디렉토리: {normal_wav_dir}")

    # 🔍 .wav 파일만 추출
    phishing_files = [phishing_wav_dir / f for f in os.listdir(phishing_wav_dir) if f.endswith(".wav")]
    normal_files = [normal_wav_dir / f for f in os.listdir(normal_wav_dir) if f.endswith(".wav")]

    print(f"🔎 phishing 파일 수: {len(phishing_files)}개")
    print(f"🔎 normal 파일 수: {len(normal_files)}개")

    # 🔁 메타데이터 CSV 파일 열기
    with open(output_csv_path, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["filename", "label", "path", "duration_sec"])  # 추가 정보: duration

        # 📦 개별 파일 처리 함수 정의
        def process_and_save(file_path, label, new_filename):
            try:
                # 1️⃣ 오디오 로딩 및 16kHz/mono 변환
                y, _ = librosa.load(file_path, sr=sample_rate, mono=True)

                # 2️⃣ 저장할 경로 지정
                output_path = output_dir / new_filename

                # 3️⃣ wav 저장
                sf.write(output_path, y, sample_rate)

                # 4️⃣ 길이(초) 계산
                duration = round(len(y) / sample_rate, 2)

                return output_path, duration

            except Exception as e:
                print(f"⚠️ 변환 실패: {file_path.name} - {e}")
                return None, None

        # 🟠 피싱 오디오 처리
        print("\n📤 피싱 오디오 처리 시작...")
        for path in phishing_files:
            name = f"phishing_{path.name}"
            out_path, dur = process_and_save(path, 1, name)
            if out_path:
                writer.writerow([name, 1, str(out_path), dur])
                print(f"✅ 저장 완료 (phishing): {name} ({dur} sec)")

        # 🔵 일반 오디오 처리
        print("\n📤 일반 오디오 처리 시작...")
        for path in normal_files:
            name = f"normal_{path.name}"
            out_path, dur = process_and_save(path, 0, name)
            if out_path:
                writer.writerow([name, 0, str(out_path), dur])
                print(f"✅ 저장 완료 (normal): {name} ({dur} sec)")

    print(f"\n📄 메타데이터 저장 완료: {output_csv_path}")


(2) 전처리 실행 코드

In [5]:
from pathlib import Path

# 📁 프로젝트 루트 경로 (로컬 Windows 환경)
project_path = Path("D:/2025_work/2025_VoicePhshing_Detection_Model")

# 📂 원본 WAV 파일 경로
phishing_wav_dir = project_path / "dataset" / "phishing_dataset" / "FSS_voicephishing_data" / "wav"
normal_wav_dir = project_path / "dataset" / "normal_dataset" / "wav"

# 📂 전처리 결과 저장 경로
output_dir = project_path / "dataset" / "processed_wav"
output_csv = project_path / "dataset" / "processed_metadata.csv"

# 🧼 전처리 함수 호출
preprocess_wav_files_without_padding(
    phishing_wav_dir=phishing_wav_dir,
    normal_wav_dir=normal_wav_dir,
    output_dir=output_dir,
    output_csv_path=output_csv,
    sample_rate=16000
)


📂 phishing 디렉토리: D:\2025_work\2025_VoicePhshing_Detection_Model\dataset\phishing_dataset\FSS_voicephishing_data\wav
📂 normal 디렉토리: D:\2025_work\2025_VoicePhshing_Detection_Model\dataset\normal_dataset\wav
🔎 phishing 파일 수: 495개
🔎 normal 파일 수: 619개

📤 피싱 오디오 처리 시작...


C:\downloads\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


✅ 저장 완료 (phishing): phishing_기타_10번_3차례 신고된 남성 전화금융사기범 (음성_10).wav (184.85 sec)
✅ 저장 완료 (phishing): phishing_기타_11번수정.wav (319.21 sec)
✅ 저장 완료 (phishing): phishing_기타_12번_3차례 신고된 남성 전화금융사기범 (음성_12).wav (143.15 sec)
✅ 저장 완료 (phishing): phishing_기타_13번수정.wav (177.11 sec)
✅ 저장 완료 (phishing): phishing_기타_14번수정.wav (163.18 sec)
✅ 저장 완료 (phishing): phishing_기타_15master.wav (178.3 sec)
✅ 저장 완료 (phishing): phishing_기타_15번_3차례 신고된 남성 전화금융사기범 (음성_15).wav (179.86 sec)
✅ 저장 완료 (phishing): phishing_기타_16번수정.wav (164.59 sec)
✅ 저장 완료 (phishing): phishing_기타_17번수정.wav (182.36 sec)
✅ 저장 완료 (phishing): phishing_기타_2번_3차례 신고된 여성 전화금융사기범 (음성_2).wav (206.12 sec)
✅ 저장 완료 (phishing): phishing_기타_3차-(1).16차례 신고된 남성 전화금융사기범 음성.wav (370.4 sec)
✅ 저장 완료 (phishing): phishing_기타_3차-(10).4차례 신고된 남성 전화금융사기범 음성 2.wav (384.9 sec)
✅ 저장 완료 (phishing): phishing_기타_3차-(11).4차례 신고된 남성 전화금융사기범 음성 3.wav (397.77 sec)
✅ 저장 완료 (phishing): phishing_기타_3차-(12).4차례 신고된 남성 전화금융사기범 음성 4.wav (311.04 sec)
✅ 저장 완료 (phishing): phishing_기